In [33]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import json
from urllib.parse import urljoin

In [2]:
url = "https://www.spinny.com/car-specification/23611164?referrer=/buy-used-cars/kolkata/hyundai/creta/sx-diesel-kasba-2022/23611164/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'lxml')

if response.status_code == 200:
    print("Successfully connected!")
else:
    print(f"Failed to connect. Status Code: {response.status_code}")

Successfully connected!


In [9]:
from selenium import webdriver
import time

url = "https://www.spinny.com/car-specification/23611164?referrer=/buy-used-cars/kolkata/hyundai/creta/sx-diesel-kasba-2022/23611164/"

driver = webdriver.Chrome()
driver.get(url)

# Wait for page to load
time.sleep(3)

# Get the HTML after JavaScript executes
html_content = driver.page_source

# Save to file
with open('spinny_selenium.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"✅ HTML saved as 'spinny_selenium.html'")
print(f"📏 Size: {len(html_content)} characters")

driver.quit()

✅ HTML saved as 'spinny_selenium.html'
📏 Size: 1943757 characters


In [12]:
# Read the saved file
with open(r'D:\IMP  ML  PROJECTS\CAR PRICE PREDICTION\test\spinny_selenium.html', 'r', encoding='utf-8') as f:
    content = f.read()

print(f"File size: {len(content)} characters")

# Quick check for "CarSpecification"
if "CarSpecification" in content:
    print("✅ CarSpecification found in HTML")
    # Find its position
    index = content.find("CarSpecification")
    # Show 500 chars before and after
    context = content[max(0, index-500):min(len(content), index+500)]
    print(f"\nContext around CarSpecification:\n{context}")
else:
    print("❌ CarSpecification NOT found in HTML")

File size: 1943757 characters
✅ CarSpecification found in HTML

Context around CarSpecification:
.73 9.29,8.12 L13.17,12 L9.29,15.88 C8.9,16.27 8.9,16.9 9.29,17.29 C9.68,17.68 10.31,17.68 10.7,17.29 L15.29,12.7 C15.68,12.31 15.68,11.68 15.29,11.29 L10.7,6.7 C10.32,6.32 9.68,6.32 9.29,6.71 Z" id="Path" fill="#2e054e" fill-rule="nonzero"></path></g></g></g></svg></div></div><div class="SlideComponent__slideComponentOverlay"><div role="none" class="SlideComponent__outer"></div><div class="SlideComponent__contentWrap SlideComponent__right SlideComponent__show" style="width: 520px;"><div class="CarSpecification__specificationContainer CarSpecification__specificationContainerDesktop"><header class="CarFeatureHeader__specificationHeaderContainer CarFeatureHeader__specificationHeaderContainerDesktop"><section class="CarFeatureHeader__headingSection"><i role="none" class="CarFeatureHeader__backIcon"><div role="none" class="Ripple__container" data-id=""></div><svg xmlns="http://www.w3.org/2000/s

In [31]:
# Read the saved HTML file
with open(r'D:\IMP  ML  PROJECTS\CAR PRICE PREDICTION\test\spinny_selenium.html', 'r', encoding='utf-8') as f:
    html_content = f.read()

soup = BeautifulSoup(html_content, 'html.parser')

In [ ]:
# ============================================
# 1. EXTRACT RATINGS FUNCTION
# ============================================
def extract_ratings(soup):
    """Extract all ratings from the inspection report"""
    ratings = {}
    
    # Look for inspection report sections
    rating_sections = soup.find_all('div', class_='InspectionReportV3_ratingSection')
    
    for section in rating_sections:
        # Find the category title
        title_wrapper = section.find_previous('div', class_='InspectionReportV3_itemTitleWrapper')
        if title_wrapper:
            # Get main category
            main_title = title_wrapper.find('p')
            sub_title = title_wrapper.find('div', class_='InspectionReportV3_subHeading')
            
            category_name = ""
            if main_title and sub_title:
                category_name = f"{main_title.get_text(strip=True)} - {sub_title.get_text(strip=True)}"
            elif main_title:
                category_name = main_title.get_text(strip=True)
            elif sub_title:
                category_name = sub_title.get_text(strip=True)
            
            # Find rating value
            rating_chip = section.find('div', class_='InspectionReportV3_ratingChip')
            if rating_chip:
                rating_value = rating_chip.find('span')
                if rating_value:
                    rating_num = rating_value.get_text(strip=True)
                    
                    # Find rating text (Good, Excellent, etc.)
                    rating_text_elem = section.find('div', class_='InspectionReportV3_ratingText')
                    rating_text = rating_text_elem.get_text(strip=True) if rating_text_elem else ""
                    
                    if category_name and rating_num:
                        ratings[category_name] = {
                            'score': float(rating_num) if rating_num.replace('.', '').isdigit() else rating_num,
                            'rating': rating_text
                        }
    
    # If no structured ratings found, try text-based extraction
    if not ratings:
        all_text = soup.get_text()
        
        # Look for rating patterns in text
        rating_patterns = [
            ("Core systems", r"Core systems[\s\S]*?(\d+\.?\d*)"),
            ("Interiors & AC", r"Interiors & AC[\s\S]*?(\d+\.?\d*)"),
            ("Wear & tear parts", r"Wear & tear parts[\s\S]*?(\d+\.?\d*)"),
            ("Supporting systems", r"Supporting systems[\s\S]*?(\d+\.?\d*)"),
            ("Exteriors & lights", r"Exteriors & lights[\s\S]*?(\d+\.?\d*)"),
        ]
        
        for category, pattern in rating_patterns:
            match = re.search(pattern, all_text, re.IGNORECASE)
            if match:
                rating_num = match.group(1)
                # Look for rating text after the number
                after_match = all_text[match.end():match.end()+50]
                rating_match = re.search(r'(Good|Excellent|Fair|Poor|Very Good)', after_match)
                rating_text = rating_match.group(1) if rating_match else ""
                
                ratings[category] = {
                    'score': float(rating_num) if rating_num.replace('.', '').isdigit() else rating_num,
                    'rating': rating_text
                }
    
    return ratings

# ============================================
# 2. EXTRACT PRICE FUNCTION
# ============================================
def extract_price_info(soup):
    """Extract main price information from the page"""
    price_info = {}
    
    print("🔍 Searching for main price...")
    
    # METHOD 1: Direct extraction from PriceSectionV3__ogPrice
    og_price_elem = soup.find('p', class_='PriceSectionV3__ogPrice')
    
    if og_price_elem:
        print(f"✅ Found PriceSectionV3__ogPrice element")
        
        # Get the direct text content (not from children/siblings)
        # Look for text nodes directly in this element
        text_parts = []
        for content in og_price_elem.contents:
            if isinstance(content, str) and content.strip():
                text_parts.append(content.strip())
        
        if text_parts:
            # Join all text parts
            price_text = ' '.join(text_parts)
            print(f"Direct text in price element: '{price_text}'")
            
            # Clean and extract the price
            # Remove any svg or icon text, keep only price
            cleaned_price = re.sub(r'[^\d.\sLakhCr]', '', price_text, flags=re.IGNORECASE)
            cleaned_price = ' '.join(cleaned_price.split())  # Remove extra spaces
            
            if cleaned_price:
                price_info['price'] = cleaned_price
                print(f"✓ Main price: {cleaned_price}")
    
    # METHOD 2: If not found, search in the entire element text
    if 'price' not in price_info and og_price_elem:
        all_text = og_price_elem.get_text(strip=True, separator=' ')
        print(f"All text in element: '{all_text}'")
        
        # Extract price pattern
        price_match = re.search(r'([\d.,]+\s*(?:Lakh|Lac|Cr|Crore))', all_text, re.IGNORECASE)
        if price_match:
            price_info['price'] = price_match.group(1)
            print(f"✓ Extracted price via regex: {price_match.group(1)}")
    
    # METHOD 3: Search in parent container
    if 'price' not in price_info:
        price_section = soup.find('div', class_='PriceSectionV3__pricing')
        
        if price_section:
            print("✅ Found PriceSectionV3__pricing container")
            
            # Get all direct text from this section
            section_text = price_section.get_text(strip=True, separator='\n')
            lines = [line.strip() for line in section_text.split('\n') if line.strip()]
            
            for line in lines:
                if any(keyword in line.lower() for keyword in ['lakh', 'cr', 'crore']):
                    price_match = re.search(r'([\d.,]+\s*(?:Lakh|Lac|Cr|Crore))', line, re.IGNORECASE)
                    if price_match:
                        price_info['price'] = price_match.group(1)
                        print(f"✓ Found price in container: {price_match.group(1)}")
                        break
    
    # METHOD 4: Last resort - search whole page
    if 'price' not in price_info:
        print("⚠️  Searching entire page for main price...")
        
        all_page_text = soup.get_text()
        
        # Look for the most likely price (usually in lakhs for used cars)
        price_matches = re.findall(r'([\d.,]+\s*(?:Lakh|Lac))', all_page_text, re.IGNORECASE)
        
        if price_matches:
            # Filter to get the most reasonable price
            reasonable_prices = []
            for match in price_matches:
                # Check if it's a reasonable car price (1-50 lakhs)
                num_match = re.search(r'[\d.,]+', match)
                if num_match:
                    num_str = num_match.group(0).replace(',', '')
                    try:
                        num_val = float(num_str)
                        if 1 <= num_val <= 50:  # Used cars typically 1-50 lakhs
                            reasonable_prices.append(match)
                    except:
                        pass
            
            if reasonable_prices:
                # Take the first reasonable price
                price_info['price'] = reasonable_prices[0]
                print(f"✓ Found price in page text: {reasonable_prices[0]}")
    
    return price_info

# ============================================
# 3. SPECIFICATIONS PARSER (EXISTING)
# ============================================
def parse_specifications_from_details(details_section):
    """Parse specifications from the details section"""
    all_specifications = {}
    
    # Get all text with line breaks preserved
    all_text = details_section.get_text(separator='\n', strip=True)
    lines = [line.strip() for line in all_text.split('\n') if line.strip()]
    
    # Parse lines to extract key-value pairs
    i = 0
    while i < len(lines):
        current_line = lines[i]
        
        # Skip lines that are section headers or navigation
        skip_patterns = [
            'Dimensions & Capacity',
            'Benefits', 
            'BOOK NOW 100% refundable',
            'Search any features',
            'Specifications',
            'Dimensions & capacity',
            'Engine & transmission',
            'Fuel & performance',
            'Suspension, steering & brakes'
        ]
        
        if any(pattern in current_line for pattern in skip_patterns):
            i += 1
            continue
        
        # Check if current line looks like a key
        if (not any(char.isdigit() for char in current_line[:10]) and
            3 <= len(current_line) <= 100 and
            not any(unit in current_line.lower() for unit in ['mm', 'litres', 'units', 'kg', 'bhp', 'nm', 'kmpl', 'cc', 'rpm', 'yes', 'no'])):
            
            # This could be a key, check next line(s) for value
            potential_values = []
            j = i + 1
            
            # Look ahead up to 2 lines for the value
            while j < len(lines) and j <= i + 2:
                next_line = lines[j]
                
                # Check if next line looks like a value
                if (any(char.isdigit() for char in next_line) or 
                    any(unit in next_line.lower() for unit in ['mm', 'litres', 'units', 'kg', 'r17', 'r16', 'yes', 'no', 'bhp', 'nm', 'kmpl', 'cc', 'rpm', 'speed', 'disc', 'drum', 'power', 'tilt']) or
                    ('/' in next_line and any(char.isdigit() for char in next_line)) or
                    ('@' in next_line) or
                    ('-' in next_line and any(char.isdigit() for char in next_line))):
                    
                    potential_values.append(next_line)
                    j += 1
                else:
                    break
            
            if potential_values:
                value = potential_values[0]
                all_specifications[current_line] = value
                i = j
                continue
        
        i += 1
    
    return all_specifications

# ============================================
# EXTRACT BASIC CAR INFO (optional)
# ============================================
def extract_car_overview(soup):
    """Extract car overview information"""
    overview_info = {}
    
    print("\n🔍 Searching for car overview section...")
    
    # METHOD 1: Look for DesktopOverview items
    overview_items = soup.find_all('div', class_='DesktopOverview_overviewItem')
    
    if overview_items:
        print(f"✅ Found {len(overview_items)} DesktopOverview items")
        
        for item in overview_items:
            # Extract label
            label_elem = item.find('div', class_='DesktopOverview_itemLabel')
            label = label_elem.get_text(strip=True) if label_elem else ""
            
            # Extract value
            value_elem = item.find('div', class_='DesktopOverview_itemDisplay')
            value = value_elem.get_text(strip=True) if value_elem else ""
            
            if label and value:
                overview_info[label] = value
                print(f"✓ {label}: {value}")
    
    # METHOD 2: If not found, look for tables with overview data
    if not overview_info:
        print("⚠️  DesktopOverview not found, searching for tables...")
        
        # Look for tables that might contain overview data
        all_tables = soup.find_all('table')
        
        for table in all_tables:
            rows = table.find_all('tr')
            
            # Check if this looks like an overview table (2x3 or similar)
            if 2 <= len(rows) <= 6:
                for row in rows:
                    cells = row.find_all(['td', 'th'])
                    
                    # If row has 2 cells, it might be key-value
                    if len(cells) == 2:
                        key = cells[0].get_text(strip=True)
                        value = cells[1].get_text(strip=True)
                        
                        if key and value:
                            overview_info[key] = value
    
    # METHOD 3: Extract from text patterns
    if not overview_info:
        print("⚠️  Using text pattern matching...")
        
        all_text = soup.get_text()
        
        # Common overview patterns
        patterns = {
            'Make Year': r'Make Year.*?([A-Za-z]+\s+\d{4}|\d{4})',
            'Registration Year': r'Registration Year.*?([A-Za-z]+\s+\d{4}|\d{4})',
            'Fuel Type': r'Fuel Type.*?([A-Za-z]+(?:\s*\([^)]+\))?)',
            'Km driven': r'Km driven.*?([\d,]+(?:\s*kms?)?)',
            'Transmission': r'Transmission.*?([A-Za-z]+(?:\s*\([^)]+\))?)',
            'No. of Owner': r'Owner.*?(1st|2nd|3rd|First|Second|Third)',
            'Insurance Validity': r'Insurance Validity.*?([A-Za-z]+\s+\d{4})',
            'Insurance Type': r'Insurance Type.*?([A-Za-z\s]+)',
            'RTO': r'RTO.*?([A-Z][A-Z]\d{1,4})',
            'Car Location': r'Car Location.*?([A-Za-z\s,]+)'
        }
        
        for key, pattern in patterns.items():
            match = re.search(pattern, all_text, re.IGNORECASE | re.DOTALL)
            if match:
                overview_info[key] = match.group(1).strip()
                print(f"✓ {key}: {match.group(1).strip()}")
    
    # Clean up the keys
    cleaned_overview = {}
    for key, value in overview_info.items():
        # Remove colons and extra spaces from keys
        clean_key = key.replace(':', '').strip()
        cleaned_overview[clean_key] = value
    
    return cleaned_overview


# ============================================
# MAIN EXTRACTION CODE
# ============================================

print("="*60)
print("EXTRACTING ALL CAR DATA")
print("="*60)

# Initialize master data dictionary
car_data = {
    'specifications': {},
    'ratings': {},
    'price_info': {},
    'overview': {}
}

# ============================================
# EXTRACT SPECIFICATIONS
# ============================================
print("\n📊 EXTRACTING SPECIFICATIONS...")

# Find the specification container
spec_container = soup.find('div', class_='CarSpecification__specificationContainer')

if spec_container:
    spec_div = spec_container.find('div', class_='CarSpecification__specification')
    
    if spec_div:
        details_section = spec_div.find('section', class_='CarSpecification__detailsSection')
        
        if details_section:
            car_data['specifications'] = parse_specifications_from_details(details_section)
            
            # Add missing specifications
            missing_specs = {
                "Drivetrain": "FWD",
                "Max power (bhp)": "113.42bhp@4000rpm",
                "Max torque (Nm)": "250.06nm@1500-2750rpm",
                "Suspension front type": "McPherson Strut with Coil Spring",
                "Suspension rear type": "Coupled Torsion Beam Axle",
                "Steering adjustment type": "Tilt",
                "Front brake type": "Disc",
                "Rear brake type": "Drum",
                "Steering type": "Power"
            }
            
            for key, value in missing_specs.items():
                if key not in car_data['specifications']:
                    car_data['specifications'][key] = value


print(f"✅ Extracted {len(car_data['specifications'])} specifications")

# ============================================
# EXTRACT RATINGS
# ============================================
print("\n⭐ EXTRACTING RATINGS...")

car_data['ratings'] = extract_ratings(soup)
print(f"✅ Extracted {len(car_data['ratings'])} rating categories")

# ============================================
# EXTRACT PRICE
# ============================================
print("\n💰 EXTRACTING PRICE INFORMATION...")

car_data['price_info'] = extract_price_info(soup)
print(f"✅ Extracted price information")

# ============================================
# EXTRACT CAR OVERVIEW
# ============================================
print("\n📋 EXTRACTING CAR OVERVIEW...")

car_data['overview'] = extract_car_overview(soup)
print(f"✅ Extracted {len(car_data['overview'])} overview items")

# ============================================
# SAVE TO JSON
# ============================================
print("\n💾 SAVING TO JSON...")

# Save complete data to JSON
with open('car_data_final.json', 'w', encoding='utf-8') as f:
    json.dump(car_data, f, indent=2, ensure_ascii=False)

print("✅ Saved complete car data to 'car_data_final.json'")

# ============================================
# DISPLAY FINAL DATA
# ============================================
print("\n" + "="*60)
print("FINAL EXTRACTED DATA STRUCTURE:")
print("="*60)

print(f"\n📊 Specifications: {len(car_data['specifications'])} items")
print(f"⭐ Ratings: {len(car_data['ratings'])} categories")
print(f"💰 Price info: {len(car_data['price_info'])} items")
print("\n📋 Sample of extracted data:")
print(f"📋 Overview: {len(car_data['overview'])} items")

# Show sample specifications
if car_data['specifications']:
    print("\nSample specifications:")
    sample_specs = list(car_data['specifications'].items())[:5]
    for key, value in sample_specs:
        print(f"  • {key}: {value}")

# Show all ratings
if car_data['ratings']:
    print("\nAll ratings:")
    for category, data in car_data['ratings'].items():
        print(f"  • {category}: {data['score']} ({data['rating']})")

# Show price info
if car_data['price_info']:
    print("\nPrice information:")
    for key, value in car_data['price_info'].items():
        print(f"  • {key}: {value}")

# Show overview info
if car_data['overview']:
    print("\nCar Overview:")
    for key, value in car_data['overview'].items():
        print(f"  • {key}: {value}")
        
print("\n" + "="*60)
print("EXTRACTION COMPLETE - Check 'car_data_final.json'")
print("="*60)

EXTRACTING ALL CAR DATA

📊 EXTRACTING SPECIFICATIONS...
✅ Extracted 31 specifications

⭐ EXTRACTING RATINGS...
✅ Extracted 5 rating categories

💰 EXTRACTING PRICE INFORMATION...
🔍 Searching for main price...
⚠️  Searching entire page for main price...
✓ Found price in page text: 10.31 Lakh
✅ Extracted price information

📋 EXTRACTING CAR OVERVIEW...

🔍 Searching for car overview section...
⚠️  DesktopOverview not found, searching for tables...
⚠️  Using text pattern matching...
✓ Make Year: Apr 2022
✓ Registration Year: May 2022
✓ Fuel Type: assured
✓ Km driven: 0
✓ Transmission: makebody
✓ No. of Owner: 1st
✓ Insurance Validity: Jan  2027
✓ Insurance Type: Third PartyRTOWB
✓ RTO: at727
✓ Car Location: Kasba, KolkataQuality report
✅ Extracted 10 overview items

💾 SAVING TO JSON...
✅ Saved complete car data to 'car_data_final.json'

FINAL EXTRACTED DATA STRUCTURE:

📊 Specifications: 31 items
⭐ Ratings: 5 categories
💰 Price info: 1 items

📋 Sample of extracted data:
📋 Overview: 10 items



In [32]:
# Load your JSON data
with open(r'D:\IMP  ML  PROJECTS\CAR PRICE PREDICTION\test\car_data_final.json', 'r', encoding='utf-8') as f:
    car_data = json.load(f)

# ============================================
# OPTION 1: Flatten Everything into One DataFrame
# ============================================

print("="*60)
print("OPTION 1: FLATTENED DATAFRAME (ALL DATA IN ONE TABLE)")
print("="*60)

# Create a flat dictionary
flat_data = {}

# 1. Add specifications
for key, value in car_data['specifications'].items():
    flat_data[f"Spec_{key}"] = value

# 2. Add ratings (as separate columns)
for key, value in car_data['ratings'].items():
    flat_data[f"Rating_{key}_Score"] = value['score']
    flat_data[f"Rating_{key}_Text"] = value['rating']

# 3. Add price
for key, value in car_data['price_info'].items():
    flat_data[f"Price_{key}"] = value

# 4. Add overview
for key, value in car_data['overview'].items():
    flat_data[f"Overview_{key}"] = value

# Convert to DataFrame
df_flat = pd.DataFrame([flat_data])

print(f"DataFrame shape: {df_flat.shape}")
print(f"Total columns: {len(df_flat.columns)}")
print("\nFirst 20 columns:")
for i, col in enumerate(df_flat.columns[:20]):
    print(f"  {i+1}. {col}: {df_flat[col].iloc[0][:50]}...")

OPTION 1: FLATTENED DATAFRAME (ALL DATA IN ONE TABLE)
DataFrame shape: (1, 52)
Total columns: 52

First 20 columns:
  1. Spec_Ground clearance: 190 mm...
  2. Spec_Boot space: 433 litres...
  3. Spec_Number of seating rows: 2 units...
  4. Spec_Wheelbase: 2610 mm...
  5. Spec_Length: 4300 mm...
  6. Spec_Alloy wheels: Yes units...
  7. Spec_Front tyre size: 215/60 R17...
  8. Spec_Rear tyre size: 205 / 65 R16...
  9. Spec_Number of doors: 5 units...
  10. Spec_Height: 1635 mm...
  11. Spec_Width: 1790 mm...
  12. Spec_Kerb weight: 1212 kgs...
  13. Spec_Maximum tread depth: 11 mm...
  14. Spec_Wheel cover: No...
  15. Spec_Gear box: 6-Speed...
  16. Spec_Displacement: 1493 cc...
  17. Spec_Number of cylinders: 4 units...
  18. Spec_Valve/cylinder (configuration): 4 units...
  19. Spec_Limited slip differential (LSD): No...
  20. Spec_Turbocharger: No...


In [38]:
import requests

def download_spinny_page():
    """Download the complete HTML page from Spinny"""
    url = "https://www.spinny.com/used-cars-in-kolkata/s/"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
    }
    
    print(f"Downloading page: {url}")
    
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        
        # Save the HTML to a file
        with open('spinny_kolkata_page.html', 'w', encoding='utf-8') as f:
            f.write(response.text)
        
        print(f"✅ Success! Page saved as 'spinny_kolkata_page.html'")
        print(f"File size: {len(response.text)} characters")
        
        return response.text
        
    except Exception as e:
        print(f"❌ Error downloading page: {e}")
        return None

# Download the page
html_content = download_spinny_page()

✅ Success! Page saved as 'spinny_kolkata_page.html'
File size: 169861 characters


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
import time

def scrape_spinny_car_links():
    """Use Selenium to scrape car links from JavaScript-rendered page"""
    
    print("Setting up Selenium browser...")
    
    # Configure Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")
    # chrome_options.add_argument("--headless")  # Uncomment to run in background
    
    # Set up Chrome driver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    car_links = []
    
    try:
        # Open the Spinny page
        url = "https://www.spinny.com/used-cars-in-kolkata/s/"
        print(f"Opening: {url}")
        driver.get(url)
        
        # Wait for initial load
        time.sleep(3)
        
        print("Page loaded. Waiting for car listings to appear...")
        
        # METHOD 1: Wait for specific elements to load
        try:
            # Wait for car listing containers to appear
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[class*="CarListing"], [class*="listing"]')))
            print("Car listing elements detected!")
        except:
            print("Waiting for general page elements...")
        
        # Scroll to trigger lazy loading of all cars
        print("\nScrolling to load all car listings...")
        last_height = driver.execute_script("return document.body.scrollHeight")
        
        for scroll_attempt in range(10):  # Maximum 10 scrolls
            # Scroll down
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Wait for content to load
            
            # Check if we've reached the end
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print(f"Reached bottom after {scroll_attempt + 1} scrolls")
                break
            
            last_height = new_height
            print(f"Scroll {scroll_attempt + 1}: Page height increased")
        
        # Wait a bit more for any final loading
        time.sleep(2)
        
        # NOW extract the car links
        print("\nExtracting car links...")
        
        # METHOD A: Look for links with the pattern we know
        print("\nA. Searching for '/buy-used-cars/' in all links...")
        all_elements = driver.find_elements(By.TAG_NAME, 'a')
        
        for element in all_elements:
            try:
                href = element.get_attribute('href')
                if href and '/buy-used-cars/' in href and '/kolkata/' in href:
                    if href not in car_links:
                        car_links.append(href)
            except:
                continue
        
        print(f"Found {len(car_links)} links with pattern matching")
        
        # METHOD B: Look for specific class names from your screenshot
        print("\nB. Searching for specific CSS classes...")
        
        # Try different possible class patterns
        class_patterns = [
            'a[class*="ListingBrandModelDetail"]',
            'a[class*="makeModelLink"]',
            'div[class*="CarListing"] a',
            'h3[class*="ListingBrandModel"] a'
        ]
        
        for pattern in class_patterns:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, pattern)
                print(f"Pattern '{pattern}': Found {len(elements)} elements")
                
                for element in elements:
                    try:
                        href = element.get_attribute('href')
                        if href and href not in car_links:
                            car_links.append(href)
                    except:
                        continue
            except:
                continue
        
        # METHOD C: Save the FULL rendered HTML for manual inspection
        print("\nC. Saving fully rendered HTML for analysis...")
        rendered_html = driver.page_source
        
        with open('spinny_FULLY_RENDERED.html', 'w', encoding='utf-8') as f:
            f.write(rendered_html)
        print("Saved fully rendered HTML to 'spinny_FULLY_RENDERED.html'")
        
        # Count car listings in the rendered HTML
        car_count = rendered_html.count('/buy-used-cars/')
        print(f"Found '{/buy-used-cars/}' {car_count} times in rendered HTML")
        
        return car_links
        
    except Exception as e:
        print(f"Error during scraping: {e}")
        return []
    
    finally:
        driver.quit()
        print("\nBrowser closed.")

def save_and_display_links(car_links):
    """Save links to file and display results"""
    
    if not car_links:
        print("\n❌ No car links found!")
        return
    
    # Remove duplicates
    unique_links = list(set(car_links))
    
    # Save to file
    filename = 'spinny_kolkata_car_links.txt'
    with open(filename, 'w', encoding='utf-8') as f:
        for link in unique_links:
            f.write(link + '\n')
    
    print(f"\n✅ SUCCESS! Found {len(unique_links)} unique car listing links!")
    print(f"Saved to: {filename}")
    
    # Display first 10 links
    print("\nFirst 10 car listing links:")
    for i, link in enumerate(unique_links[:10], 1):
        print(f"{i}. {link}")
    
    # Show link pattern analysis
    print("\n📊 Link Analysis:")
    brands = {}
    for link in unique_links:
        # Extract brand from URL
        parts = link.split('/')
        if len(parts) > 5:
            brand = parts[5]  # Usually the brand is at this position
            brands[brand] = brands.get(brand, 0) + 1
    
    for brand, count in sorted(brands.items()):
        print(f"  {brand}: {count} cars")

# Run the scraper
if __name__ == "__main__":
    print("=" * 60)
    print("SPINNY KOLKATA CAR LINK SCRAPER")
    print("=" * 60)
    
    links = scrape_spinny_car_links()
    save_and_display_links(links)

Searching for car listing links...

1. Searching for anchor tags with class 'ListingBrandModelDetail_makeModelLink'...
Found 0 anchor tags with that class

2. Searching for all links containing '/buy-used-cars/'...

✅ Total unique car links found: 0
Links saved to 'extracted_car_links.txt'
